In [4]:
search_for = 124
start_from = 0
threads = 10
thread = 7
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0065670013427734375
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 75
all cases: 621252555
type: [1, 1, 1, 1, 124] 124
cases of this type: 236421376
100000 26.88395822509609
200000 51.38011461261401
300000 55.216526466392374
400000 46.86006572729975
500000 57.029321600536804
600000 55.45516913260072
700000 53.44092252481593
800000 48.41213576397078
900000 54.41935286503261
1000000 19.683342472414935
1100000 54.78655757901688
1200000 42.85818318792447
1300000 45.78246803942946
1400000 43.10784881059728
1500000 46.26119635571932
1600000 46.462175629441624
1700000 43.25061698293843
1800000 47.36995054263625
1900000 38.24266340410031
2000000 21.9399032632819
2100000 46.104994474003234
2200000 42.263375021285015
2300000 45.27972040450067
2400000 42.904682185200954
2500000 44.620103422136225
2600000 43.11259755975759
2700000 41.6344808847253
2800000 44.67532117350168
2900000 17.57308119409616
3000000 40.504998526

29700000 42.48189383829963
29800000 44.68753820722497
29900000 41.477287380289084
30000000 43.151944393107456
30100000 44.34327064439873
30200000 45.03288843444064
30300000 27.28279473643267
30400000 39.73343664066596
30500000 45.62496136256702
30600000 44.31025319575891
30700000 43.565879714297495
30800000 19.751338801437324
30900000 44.148391432106905
31000000 44.76646066299495
31100000 42.105726241507924
31200000 42.70614834214354
31300000 38.99549545209065
31400000 38.98102536840417
31500000 17.508547694008985
31600000 40.27326960334684
31700000 38.222290547202945
31800000 40.741698001605535
31900000 40.9517845058118
32000000 43.21898204261164
32100000 42.84232934138659
32200000 23.750451390313152
32300000 41.563107302867415
32400000 42.81516283334301
32500000 45.365599724859486
32600000 45.00316044122298
32700000 26.67690682739534
32800000 39.25037013077209
32900000 45.81314161861622
33000000 43.4364316640462
33100000 42.73244568943449
33200000 41.79576857588965
33300000 39.259072

59800000 38.53896136232915
59900000 37.5522101315275
60000000 38.64707019829816
60100000 15.37818344959226
60200000 39.43745156165265
60300000 37.643958991659616
60400000 38.700036306189034
60500000 38.023900070841925
60600000 20.504815005437326
60700000 40.48467640006072
60800000 41.17439612521726
60900000 41.45487958028278
61000000 43.70959445318407
61100000 42.24467878134931
61200000 41.46451691911321
61300000 40.45216833607254
61400000 40.09666042151046
61500000 32.657357804117304
61600000 25.222350421737666
61700000 36.095086487169986
61800000 37.63077133241245
61900000 36.14692100415313
62000000 17.838344192950043
62100000 35.33513658562859
62200000 37.73592557755721
62300000 35.37082064904805
62400000 37.1231890431449
62500000 22.264893060594243
62600000 38.29380859764134
62700000 39.005969042189236
62800000 40.43736349209248
62900000 41.13029123137851
63000000 41.92210346672101
63100000 42.13645037926393
63200000 43.06774879199632
63300000 40.40716664165472
63400000 40.72904941

89900000 37.95819019230378
90000000 36.968893348774344
90100000 36.29773310721368
90200000 36.98828156226707
90300000 35.72045955040365
90400000 17.46232313668368
90500000 34.40732669887324
90600000 16.455560635365156
90700000 33.711017208605725
90800000 28.281015650593726
90900000 27.419089039863557
91000000 36.820735272272465
91100000 35.41401350863873
91200000 36.560598725555344
91300000 36.57728872241466
91400000 38.583854906747554
91500000 39.59423743411732
91600000 39.20568415108053
91700000 39.03090166600807
91800000 37.134912023828974
91900000 34.94835594344695
92000000 36.61626686187797
92100000 35.90489237624286
92200000 35.06911382012284
92300000 16.805593109206104
92400000 36.71079837200275
92500000 17.233259400390576
92600000 31.834799253439066
92700000 24.4740649979832
92800000 29.45594934051905
92900000 35.73218529830979
93000000 36.05611497899902
93100000 35.79338640574399
93200000 35.76338576359011
93300000 37.67843101195815
93400000 39.46635023138695
93500000 40.03731

119200000 3.8339353423627944
119300000 3.7855567380366026
119400000 3.766665146091044
119500000 3.7817897788486614
119600000 3.7968974523087806
119700000 3.6881430606966616
119800000 3.7060567293993865
119900000 3.7710866978163424
120000000 3.7664090119443734
120100000 3.741184617678907
120200000 3.7504953262791503
120300000 3.700770249771446
120400000 3.8154042645584942
120500000 3.75690576955734
120600000 3.7050902311853373
120700000 3.740680240217401
120800000 3.7062848204020793
120900000 3.7314136584006152
121000000 3.7337424042627285
121100000 3.7635934758078884
121200000 3.8075736757767498
121300000 3.7245090173674553
121400000 3.779612686399172
121500000 3.755564975203613
121600000 3.7458601447023785
121700000 3.733984282869117
121800000 3.7788307544709743
121900000 3.7004867256612743
122000000 3.690072409239474
122100000 3.6537881404255628
122200000 3.6754788945801296
122300000 3.670550730985344
122400000 3.671266425859282
122500000 3.6904320375842055
122600000 3.67711176970148

...

493100000 4.647607770380434
493200000 4.495238461931537
493300000 4.150825640824662
493400000 1.511103112784707
493500000 3.785280715169996
493600000 3.377549295382427
493700000 3.0418466169222786
493800000 3.820885099538684
493900000 3.985086557343129
494000000 4.170153693336218
494100000 4.913639280269851
494200000 4.022194078050074
494300000 2.6940663017326263
494400000 2.689592725442251
494500000 3.547563191171408
494600000 2.5382281923387118
494700000 3.8618638462075294
494800000 4.019646031452057
494900000 4.163593747682356
495000000 4.187754779846022
495100000 4.558369514636202
495200000 4.173755374756608
495300000 1.2648588266744913
495400000 3.645967665515393
495500000 2.3396756447892124
495600000 3.835097481155455
495700000 3.9081238356371752
495800000 4.031445168406083
495900000 3.8778864784822957
496000000 4.014460722900437
496100000 4.508186427360005
496200000 2.6803997942438325
496300000 2.513959200286372
496400000 2.7078574925627046
496500000 3.502494695246279
496600000 

521100000 3.1230166365274887
521200000 3.0220823325064643
521300000 3.12491189407096
521400000 3.094627471910248
521500000 3.2764323668136295
521600000 3.3166503096366555
521700000 3.0830678521990778
521800000 3.290905333318263
521900000 3.2899304523554083
522000000 3.3811586634991864
522100000 3.24356846777008
522200000 3.2774986919389333
522300000 3.2657415028502843
522400000 3.3577944828526776
522500000 3.3185681700588465
522600000 3.2429212275398074
522700000 3.3025461312694415
522800000 3.3632467410538958
522900000 3.214242611715605
523000000 3.2981656598543125
523100000 3.131488110751255
523200000 3.318850111176252
523300000 3.2442694220078683
523400000 3.211780618861447
523500000 2.966876373392969
523600000 2.9858813640029003
523700000 3.011872792932603
523800000 3.070519228902628
523900000 3.127080248012006
524000000 3.169396923222048
524100000 3.0217245018865664
524200000 3.288924142655591
524300000 3.0577351757004325
524400000 3.7694246483971177
524500000 3.470475547234628
52

547900000 3.7995944352789492
548000000 2.1821502999663718
548100000 3.3837549373715614
548200000 3.8697666624471014
548300000 1.219754175360153
548400000 0.5740084344646235
548500000 0.5715705818479028
548600000 0.5702736178899672
548700000 0.5626209956050814
548800000 0.5609137385292782
548900000 0.5607799712009993
549000000 0.5597545968578657
549100000 0.5603155567118326
549200000 0.5584287669497662
549300000 0.5604983885148962
549400000 0.5599047053576444
549500000 0.5595060030411216
549600000 0.5575210046613812
549700000 0.5607789934947424
549800000 0.5569182050243185
549900000 0.5763314271218181
550000000 0.5570852370718088
550100000 0.5565667114828593
550200000 0.5590771622506777
550300000 0.5533153846225738
550400000 0.5435530182094707
550500000 0.5471068746961355
550600000 0.5668912875663704
550700000 4.556920171984864
550800000 4.205121516458054
550900000 1.5773820255186324
551000000 4.336605917462167
551100000 4.48173537561512
551200000 3.4409804580696224
551300000 2.74208641

576300000 1.443031197824965
576400000 1.2991074549827641
576500000 1.1218041274593835
576600000 0.7578280255690217
576700000 1.2775139112468992
576800000 1.5176949882659083
576900000 1.077110987674147
577000000 1.2967251915111973
577100000 0.5700900070823265
577200000 1.3785029117369751
577300000 1.1695632952300608
577400000 0.8356362954816785
577500000 1.2022058537142277
577600000 0.6263478025124105
577700000 1.4711002855249942
577800000 1.1633575757146777
577900000 1.1484456261433826
578000000 0.7311429847966565
578100000 0.9868952835419973
578200000 1.4854845742581555
578300000 0.6179644794279204
578400000 1.0011274400624037
578500000 0.672476520097345
578600000 1.1890837614395255
578700000 1.3498311225294075
best so far: 0
type: [1, 2, 1, 31, 2] 124
cases of this type: 953312
578800000 0.35943282464342313
578900000 0.2020412422972123
579000000 0.27185255559123556
579100000 0.2611551563323074
579200000 0.2665613010571268
579300000 0.2682678441735506
579400000 0.2676451581555175
5795

599700000 0.1533213461550176
599800000 0.4076068512527943
599900000 0.7179869598853126
600000000 0.6419087385252417
600100000 0.6099209498568575
600200000 0.5367919012167388
600300000 0.5853131956032415
600400000 0.44268626936032046
600500000 0.7389980773030718
600600000 0.5232331092432638
600700000 0.5705157539792657
600800000 0.7779310806540548
600900000 0.44434657543635364
601000000 0.5962290156266391
601100000 0.4909282602219416
601200000 0.5748779808512131
601300000 0.5906507117926809
601400000 0.42793285169630907
601500000 0.6871734402641952
601600000 0.5126075977172454
601700000 0.6712532076126834
601800000 0.5216475582190454
601900000 0.5667358774770995
602000000 0.6915253581087855
602100000 0.48579143515448764
602200000 0.7421656883895498
602300000 0.49047497130061185
602400000 0.577333531917274
602500000 0.6424386790789399
602600000 0.48135904787246714
602700000 0.7351959017905593
602800000 0.5532274182040361
602900000 0.6131163611016472
603000000 0.5303017906697393
603100000